In [1]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima  # For automatic ARIMA model selection
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load the dataset
df = pd.read_csv(r"C:\Users\4t4\OneDrive\Desktop\time series for beds\holt's-W\WARD_daily.csv")

# Convert 'Date' to datetime format for proper time series analysis
df['Date'] = pd.to_datetime(df['Date'])

# Get a list of unique wards for individual analysis
wards = df['WARD'].unique()

# Initialize a DataFrame to store evaluation results for all wards
all_evaluation_results = pd.DataFrame()

# Wards to exclude from analysis due to data quality issues or irrelevance
excluded_wards = ["KHCC-ADULT DAY CASE NZR", "KHCC-EMERGENCY KSB ROOM", "KHCC-BMT DAY CASE"]

# Iterate through each ward for individual forecasting
for ward in wards:
    if ward in excluded_wards:
        print(f"Skipping ward: {ward}")
        continue

    print(f"\nProcessing ward: {ward}")

    # Filter the data for the current ward
    df_ward = df[df['WARD'] == ward][['Date', 'TotalBedsUsed']].copy()

    # Check if there's sufficient data for analysis
    if len(df_ward) < 187:
        print(f"Not enough data for ward {ward}. Skipping.")
        continue

    # Sort the data chronologically and set 'Date' as the index
    df_ward.sort_values('Date', inplace=True)
    df_ward.set_index('Date', inplace=True)

    # Extract 'TotalBedsUsed' for modeling
    total_beds = df_ward['TotalBedsUsed']

    # Split the data: all but last 180 days for training, last 180 days for testing
    train_data = total_beds.iloc[:-180]
    test_data = total_beds.iloc[-180:]

    # Initialize a DataFrame to store actual and forecasted values for 180 days
    forecast_results = pd.DataFrame(index=test_data.index, columns=['Actual', 'AutoSARIMA'])
    forecast_results['Actual'] = test_data

    # Fit the Auto ARIMA model on the training data
    try:
        model = auto_arima(train_data, seasonal=True, m=7, suppress_warnings=True, error_action="ignore")
    except Exception as e:
        print(f"Error fitting model for ward {ward}: {e}")
        continue

    # Perform rolling forecast for the 180-day period
    for i in range(0, 180, 7):
        # Update training data to include actual values up to the current point
        temp_train_data = total_beds.iloc[:-(180 - i)]

        # Refit the model with updated training data
        try:
            model = auto_arima(temp_train_data, seasonal=True, m=7, suppress_warnings=True, error_action="ignore")
        except Exception as e:
            print(f"Error refitting model for ward {ward}: {e}")
            continue

        # Forecast the next 7 days
        try:
            forecast = model.predict(n_periods=7)
            if len(forecast) == 7:
                forecast_results['AutoSARIMA'].iloc[i:i+7] = forecast
            else:
                print(f"Skipping forecast for ward {ward} at step {i} due to incorrect length.")
        except Exception as e:
            print(f"Error forecasting for ward {ward}: {e}")
            continue

    # Plot actual vs forecasted values for the 180-day period
    fig = go.Figure()

    # Actual values
    fig.add_trace(go.Scatter(
        x=forecast_results.index, y=forecast_results['Actual'], mode='lines+markers',
        name='Actual Data', line=dict(color='black')
    ))

    # Forecasted values
    fig.add_trace(go.Scatter(
        x=forecast_results.index, y=forecast_results['AutoSARIMA'], mode='lines+markers',
        name='Auto SARIMA Forecast', line=dict(color='red', dash='dash')
    ))

    # Customize plot layout
    fig.update_layout(
        title=f"Actual vs Forecast (180 Days) for Ward: {ward}",
        xaxis_title='Date',
        yaxis_title='Total Beds Used',
        showlegend=True
    )

    fig.show()

    # Calculate and store MSE and MAE for the forecast
    try:
        valid_forecasts = forecast_results.dropna()
        mse_results = pd.DataFrame({
            'Model': ['AutoSARIMA'],
            'MSE': [mean_squared_error(valid_forecasts['Actual'], valid_forecasts['AutoSARIMA'])],
            'MAE': [mean_absolute_error(valid_forecasts['Actual'], valid_forecasts['AutoSARIMA'])]
        })

        # Add the ward name and append to the overall results
        mse_results['Ward'] = ward
        all_evaluation_results = pd.concat([all_evaluation_results, mse_results], ignore_index=True)
    except Exception as e:
        print(f"Error calculating MSE and MAE for ward {ward}: {e}")

# Save evaluation results for all wards to a CSV file
all_evaluation_results.to_csv('evaluation_results_all_wards_autosarima.csv', index=False)
print("\nEvaluation results saved to 'evaluation_results_all_wards_autosarima.csv'")



Processing ward: KHCC-10H-BMT


c:\Users\4t4\anaconda3\envs\lstm_myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\4t4\anaconda3\envs\lstm_myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\4t4\AppData\Local\Temp\ipykernel_22236\784830628.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast_results['AutoSARIMA'].iloc[i:i+7] = forecast
c:\Users\4t4\anaconda3\envs\lstm_myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No su

Error forecasting for ward KHCC-10H-BMT: cannot set using a slice indexer with a different length than the value



Processing ward: KHCC-10H-MEDICAL/SURGICAL


C:\Users\4t4\AppData\Local\Temp\ipykernel_22236\784830628.py:83: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\4t4\AppData\Local\Temp\ipykernel_22236\784830628.py:83: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\4t4\AppData\Local\Temp\ipykernel_22236\784830628.py:83: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\4t4\AppData\Local\Temp\ipykernel_22236\784830628.py:83: SettingWithCopyWarning:


A value is tryi

...

Skipping ward: KHCC-BMT DAY CASE
Skipping ward: KHCC-ADULT DAY CASE NZR

Processing ward: KHCC-5C-KOLAGHASSI WING


c:\Users\4t4\anaconda3\envs\lstm_myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\4t4\anaconda3\envs\lstm_myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

C:\Users\4t4\AppData\Local\Temp\ipykernel_22236\784830628.py:83: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

c:\Users\4t4\anaconda3\envs\lstm_myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Us

Error forecasting for ward KHCC-5C-KOLAGHASSI WING: cannot set using a slice indexer with a different length than the value


c:\Users\4t4\anaconda3\envs\lstm_myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

c:\Users\4t4\anaconda3\envs\lstm_myenv\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

C:\Users\4t4\AppData\Local\Temp\ipykernel_22236\784830628.py:83: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy




Evaluation results saved to 'evaluation_results_all_wards_autosarima.csv'


In [2]:
import pandas as pd
import plotly.graph_objects as go

# Load the evaluation results for Holt's and AutoARIMA models
holts_results = pd.read_csv(r"C:\Users\4t4\OneDrive\Desktop\time series for beds\holt's-W\evaluation_results_all_wards_holts.csv")
autosarima_results = pd.read_csv('evaluation_results_all_wards_autosarima.csv')

# Combine the two datasets into one
all_results = pd.concat([holts_results, autosarima_results], ignore_index=True)

# Get a list of unique wards
wards = all_results['Ward'].unique()

# Create a plot for each ward
for ward in wards:
    # Filter data for the current ward
    ward_data = all_results[all_results['Ward'] == ward]
    
    # Extract MSE and MAE data for plotting
    models = ward_data['Model'].values
    mse_values = ward_data['MSE'].values
    mae_values = ward_data['MAE'].values

    # Create the figure
    fig = go.Figure()

    # Add MSE bars
    fig.add_trace(go.Bar(
        x=models,
        y=mse_values,
        name='MSE',
        text=mse_values,
        textposition='auto',
        marker=dict(color='red')
    ))

    # Add MAE bars
    fig.add_trace(go.Bar(
        x=models,
        y=mae_values,
        name='MAE',
        text=mae_values,
        textposition='auto',
        marker=dict(color='blue')
    ))

    # Update the layout
    fig.update_layout(
        title=f'MSE and MAE Comparison for Ward: {ward}',
        xaxis_title='Model',
        yaxis_title='Error',
        barmode='group',
        showlegend=True
    )

    # Show the plot for each ward
    fig.show()
